## Attempts at constructing a 2-dimensional Mapper Graph
The Mapper Algorithm is a useful tool in data analysis to produce readable graphs representing some higher dimensional data set. However a Mapper graph, as it was constucted in *Topological Methods for the Analysis of High Dimensional Data Sets and 3D Object Recognition* [Singh, et al.],  is only a 1-dimensional graph, but we believe there is a use for a Mapper graph which can output 2-dimensional cells as well to better visualize strong relationships within the data. 

Giotto-tda already has documentation to produce a 1-dimensional Mapper graph, and so my goal is to extend this to constructing a 2-dimensional Mapper graph. 

### Overview of the Mapper Algorithm
Let $X$ be a data set which we think of as a finite compact metric space embeddable in some $\mathbb{R}^n$. For our algorithm we need the following tools:

1. A filter function $f\colon X\to \mathbb{R}^m$, where $m\leq n$. This filter function should be some function which reduces the dimension of the data set $X$. Typically $f$ is some projection of $X$ into $\mathbb{R}$ or $\mathbb{R}^2$. PCA is also a good choice of filter function. 
2. We use our filter function to view $f(X)\subset \mathbb{R}^m$ and we choose a (good open) cover $U=\{$ of $f(X)$. Currently in giotto-tda they have a `CubicalCover` which is extendable to any $\mathbb{R}^n$. One goal is to create another cover based on [Overlapping Circles](https://en.wikipedia.org/wiki/Overlapping_circles_grid).
3. After choosing a cover on our data's image $f(X)$, we choose a clustering algorithm (typically `DBSCAN` from the `sklearn` package). With this, we take the preimage of each cover set, $f^{-1}(U_\alpha)$ and use are clustering algorithm to determine individual clusters $\bigcup_{i=1}^k C_{\alpha,i}=f^{-1}(U_\alpha)$.
4. Once the clusters are determined we want to find the nerve of the of the cover $\{f^{-1}(U_\alpha)\}$ of $X$. We note that each cluster $C_{\alpha,i}$ is a node in our Mapper graph. To find edges of our nerve we compute $C_{\alpha,i}\cap C_{\beta,j}$ for all pairs of clusters. For each pair such that $C_{\alpha,i}\cap C_{\beta,j}\neq \emptyset$ we connect these with an edge. 

This algorithm constructs a Mapper graph of the dataset $X$. With the following steps we can extend Mapper to become a 2 dimensional graph:

5. For each triple of clusters we check $C_{\alpha,i}\cap C_{\beta,j}\cap C_{\gamma,k}\neq \emptyset$. Each triple with this property will add a 2-simplex $\sigma_{\alpha\beta\gamma}^{ijk}$ to our Mapper graph. 



In [2]:
import numpy as np
import plotly.graph_objects as go
import igraph as ig
from gtda.plotting import plot_point_cloud

from gtda.mapper import (
    CubicalCover,
    make_mapper_pipeline,
    Nerve,
    Projection,
    plot_static_mapper_graph
)

from sklearn import datasets
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

In [18]:
# Generate some data
cdata, c_ = datasets.make_circles(n_samples = 5000, noise = 0.05, factor=0.3, random_state=9)
mdata, m_ = datasets.make_moons(n_samples=1000, noise = 0.05, random_state=9)
sdata, s_ = datasets.make_swiss_roll(n_samples = 5000, noise = 0.05, random_state=9)

In [19]:
plot_point_cloud(sdata)

In [20]:
plot_point_cloud(mdata)

In [21]:
plot_point_cloud(cdata)

In [28]:
# Choose some hyperparameters
filter_func = Projection(columns=[0,1]) #filter function
cover = CubicalCover(n_intervals=10, overlap_frac=0.3) # Cover
clusterer = DBSCAN() # Clustering Algorithm
n_jobs=1 # Number of processors for the pipeline

# Construct a basic pipeline
pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    n_jobs = n_jobs,
    verbose=False,
)

In [26]:
# Make a mapper graph
fig = plot_static_mapper_graph(pipe,cdata)
fig.show(config={'scrollZoom':True})